In [26]:
import pandas as pd
import numpy as np
import argparse, time
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
import os, sys
from dgl.data import register_data_args, load_data
import pandas as pd
from gcn import GCN

In [27]:
def _normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def _encode_onehot(labels):
    classes = list(sorted(set(labels)))
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

def _sample_mask(idx, l):
    """Create mask."""
    mask = np.zeros(l)
    mask[idx] = 1
    return mask

def evaluate(model, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)


<b>CernDataset class is built based on the strucurue of coreDataset </b><br>
find it here: https://github.com/dmlc/dgl/blob/master/python/dgl/data/citation_graph.py <br>
The code will build a fully connected graph between layer. each node (layercluster) is fully connected to all nodes from previous layer. In other word, each node is a parent for each and every node in the next layer. 
<br><br>
<b>train_size</b> defines the propotion of nodes selected for training. <br>
<b>val_size</b> defines the propotion of nodes selected for Validation.<br>
all left (1 - train_size - val_size) is allocated for Test.(<b>test_size</b><br>
actual validation may be less than <b>val_size</b> as after creating the mask we remove the nodes allocated for train from it using these two mask commands: <br>
xor_val_train=np.bitwise_xor(val_mask,train_mask) <br>
_val_mask=np.bitwise_and(val_mask,xor_val_train)<br><br>
That means, test_size may be greater than actual "1 - train_size - val_size". 

In [94]:
class CernDataset(object):
    def getListEventLayerTrackster(self, df, event, trackster,layer):
        filtered_df = df.loc[(df['event'] == event) & (df['layer'] ==layer) & (df['trackster'] ==trackster)]
        return filtered_df

    def __init__(self, train_size=0.15, val_size=0.50):
         self.store_oct27 = pd.read_hdf("singlepi_e100GeV_pu200_oct27.h5")
         #filter out some events - to minimize the processing time during development.
         #self.store_oct27 = self.store_oct27[self.store_oct27['event'] < 5 ]
         self.store_oct27['purity']=self.store_oct27['purity'].apply(lambda x: 0 if x <=1 else 1 )
         self.df = self.store_oct27.drop(['eta','phi','layer','trckPhi','trckEn','trckEta','trckType'],1,inplace=False)
         self.train_size = train_size
         self.val_size = val_size
         self._load()
         
    def _load(self):
        
        idx_features_labels =  self.df.drop(['purity','event','trackster'],1,inplace=False)
        idx_features_labels = idx_features_labels.to_numpy()
        features = sp.csr_matrix(idx_features_labels,dtype=np.float32)
        labels = _encode_onehot(self.df[['purity']].iloc[:,0])
        self.num_labels = labels.shape[1]
        
        # build graph
        edges_flatted =[]
        for idx, row in self.store_oct27.iterrows():
            prev_layer = self.getListEventLayerTrackster(self.store_oct27, row['event'],row['trackster'],row['layer']-1)
            for jdx,row in prev_layer.iterrows():
                edges_flatted.append(jdx)
                edges_flatted.append(idx)
                
        edges= np.array(edges_flatted).reshape(len(edges_flatted) // 2,2)
        
        adj = sp.coo_matrix((np.ones(edges.shape[0]),
                             (edges[:, 0], edges[:, 1])),
                            shape=(labels.shape[0], labels.shape[0]),
                            dtype=np.float32)

        self.graph = nx.from_scipy_sparse_matrix(adj, create_using=nx.DiGraph())

        features = _normalize(features)
        self.features = np.array(features.todense())
        self.labels = np.where(labels)[1]
        #test_size = int(labels.shape[0] * 0.15)
        train_size = int(labels.shape[0] * self.train_size)
        val_size = int(labels.shape[0] * self.val_size)
        
        train_mask= np.zeros(labels.shape[0], dtype=bool)
        train_mask[:train_size] = 1
        np.random.shuffle(train_mask)
        self.train_mask = train_mask
        
        val_mask= np.zeros(labels.shape[0], dtype=bool)
        val_mask[:val_size] = 1
        np.random.shuffle(val_mask)
        xor_val_train=np.bitwise_xor(val_mask,train_mask)
        _val_mask=np.bitwise_and(val_mask,xor_val_train)
        
        self.val_mask = _val_mask.tolist()
        #all layercluster not chosen for training or validation will be added to the test 
        test_mask = np.bitwise_or(self.val_mask, self.train_mask)
        _test_mask = np.invert(test_mask)
        self.test_mask = _test_mask.tolist()
        #self.train_mask = _sample_mask(range(1500), labels.shape[0])
        #self.val_mask = _sample_mask(range(2000, 5000), labels.shape[0])
        #self.test_mask = _sample_mask(range(5000, 8000), labels.shape[0])

    def __getitem__(self, idx):
        assert idx == 0, "This dataset has only one graph"
        g = DGLGraph(self.graph)
        g.ndata['train_mask'] = self.train_mask
        g.ndata['val_mask'] = self.val_mask
        g.ndata['test_mask'] = self.test_mask
        g.ndata['label'] = self.labels
        g.ndata['feat'] = self.features
        return g
    
    def __len__(self):
        return 1


### defind the main model building function.

In [95]:
def graphCovNN(args):
    # load and preprocess dataset
    data = args.dataset
    features = torch.FloatTensor(data.features)
    labels = torch.LongTensor(data.labels)
    if hasattr(torch, 'BoolTensor'):
        train_mask = torch.BoolTensor(data.train_mask)
        val_mask = torch.BoolTensor(data.val_mask)
        test_mask = torch.BoolTensor(data.test_mask)
    else:
        train_mask = torch.ByteTensor(data.train_mask)
        val_mask = torch.ByteTensor(data.val_mask)
        test_mask = torch.ByteTensor(data.test_mask)
    in_feats = features.shape[1]
    n_classes = data.num_labels
    n_edges = data.graph.number_of_edges()
    print("""----Data statistics------'
      #Edges %d
      #Classes %d
      #Train samples %d
      #Val samples %d
      #Test samples %d""" %
          (n_edges, n_classes,
              train_mask.int().sum().item(),
              val_mask.int().sum().item(),
              test_mask.int().sum().item()))

    if args.gpu < 0:
        cuda = False
    else:
        cuda = True
        torch.cuda.set_device(args.gpu)
        features = features.cuda()
        labels = labels.cuda()
        train_mask = train_mask.cuda()
        val_mask = val_mask.cuda()
        test_mask = test_mask.cuda()

    # graph preprocess and calculate normalization factor
    g = data.graph
    # add self loop
    if args.self_loop:
        g.remove_edges_from(nx.selfloop_edges(g))
        g.add_edges_from(zip(g.nodes(), g.nodes()))
    g = DGLGraph(g)
    n_edges = g.number_of_edges()
    # normalization
    degs = g.in_degrees().float()
    norm = torch.pow(degs, -0.5)
    norm[torch.isinf(norm)] = 0
    if cuda:
        norm = norm.cuda()
    g.ndata['norm'] = norm.unsqueeze(1)

    # create GCN model
    model = GCN(g,
                in_feats,
                args.n_hidden,
                n_classes,
                args.n_layers,
                F.relu,
                args.dropout)

    if cuda:
        model.cuda()
    loss_fcn = torch.nn.CrossEntropyLoss()

    # use optimizer
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=args.lr,
                                 weight_decay=args.weight_decay)

    # initialize graph
    dur = []
    for epoch in range(args.n_epochs):
        model.train()
        if epoch >= 3:
            t0 = time.time()
        # forward
        logits = model(features)
        loss = loss_fcn(logits[train_mask], labels[train_mask])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch >= 3:
            dur.append(time.time() - t0)

        acc = evaluate(model, features, labels, val_mask)
        print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} |"
              "ETputs(KTEPS) {:.2f}". format(epoch, np.mean(dur), loss.item(),
                                             n_edges / np.mean(dur) / 1000))

    print()
    acc = evaluate(model, features, labels, test_mask)
    print("Test accuracy {:.2%}".format(acc))

In [96]:
class dummy(object):
    pass

In [97]:
#configularble paramters 
args=dummy()
args.dropout=0.3 #dropout probability
args.gpu=-1
args.lr=0.01 #learning rate
args.n_epochs=15
args.n_hidden=16 #number of hidden gcn units
args.n_layers=3
args.self_loop=False #graph self-loop 
args.weight_decay=0.0005
args.dataset = CernDataset()
graphCovNN(args)

----Data statistics------'
      #Edges 514910
      #Classes 2
      #Train samples 4841
      #Val samples 13710
      #Test samples 13723
Epoch 00000 | Time(s) nan | Loss 0.6637 |ETputs(KTEPS) nan
Epoch 00001 | Time(s) nan | Loss 0.6272 |ETputs(KTEPS) nan
Epoch 00002 | Time(s) nan | Loss 0.5939 |ETputs(KTEPS) nan
Epoch 00003 | Time(s) 0.1765 | Loss 0.5663 |ETputs(KTEPS) 2916.88
Epoch 00004 | Time(s) 0.1740 | Loss 0.5385 |ETputs(KTEPS) 2958.68
Epoch 00005 | Time(s) 0.1715 | Loss 0.5230 |ETputs(KTEPS) 3001.60
Epoch 00006 | Time(s) 0.1769 | Loss 0.5179 |ETputs(KTEPS) 2910.43
Epoch 00007 | Time(s) 0.1741 | Loss 0.5247 |ETputs(KTEPS) 2958.36
Epoch 00008 | Time(s) 0.1727 | Loss 0.5320 |ETputs(KTEPS) 2981.12
Epoch 00009 | Time(s) 0.1725 | Loss 0.5280 |ETputs(KTEPS) 2985.26
Epoch 00010 | Time(s) 0.1711 | Loss 0.5275 |ETputs(KTEPS) 3010.14
Epoch 00011 | Time(s) 0.1697 | Loss 0.5232 |ETputs(KTEPS) 3034.75
Epoch 00012 | Time(s) 0.1717 | Loss 0.5210 |ETputs(KTEPS) 2998.82
Epoch 00013 | Time(s) 

In [98]:
# try with different setup.
args1=dummy()
args1.dropout=0.5 #dropout probability
args1.gpu=-1
args1.lr=0.1 #learning rate
args1.n_epochs=10
args1.n_hidden=32 #number of hidden gcn units
args1.n_layers=3
args1.self_loop=False #graph self-loop 
args1.weight_decay=0.0005
args1.dataset = CernDataset(0.65, 0.25)

graphCovNN(args1)

----Data statistics------'
      #Edges 514910
      #Classes 2
      #Train samples 20978
      #Val samples 2809
      #Test samples 8487
Epoch 00000 | Time(s) nan | Loss 0.7216 |ETputs(KTEPS) nan
Epoch 00001 | Time(s) nan | Loss 0.9240 |ETputs(KTEPS) nan
Epoch 00002 | Time(s) nan | Loss 0.6477 |ETputs(KTEPS) nan
Epoch 00003 | Time(s) 0.2718 | Loss 0.5574 |ETputs(KTEPS) 1894.60
Epoch 00004 | Time(s) 0.2725 | Loss 0.5102 |ETputs(KTEPS) 1889.37
Epoch 00005 | Time(s) 0.2695 | Loss 0.4951 |ETputs(KTEPS) 1910.39
Epoch 00006 | Time(s) 0.2804 | Loss 0.5118 |ETputs(KTEPS) 1836.06
Epoch 00007 | Time(s) 0.2776 | Loss 0.5008 |ETputs(KTEPS) 1854.77
Epoch 00008 | Time(s) 0.2746 | Loss 0.4964 |ETputs(KTEPS) 1874.79
Epoch 00009 | Time(s) 0.2727 | Loss 0.4953 |ETputs(KTEPS) 1887.87

Test accuracy 80.45%


In [99]:
# try with different setup.
args2=dummy()
args2.dropout=0.5 #dropout probability
args2.gpu=-1
args2.lr=0.1 #learning rate
args2.n_epochs=20
args2.n_hidden=32 #number of hidden gcn units
args2.n_layers=3
args2.self_loop=False #graph self-loop 
args2.weight_decay=0.0005
args2.dataset = CernDataset(0.7, 0.18)

graphCovNN(args2)

----Data statistics------'
      #Edges 514910
      #Classes 2
      #Train samples 22591
      #Val samples 1756
      #Test samples 7927
Epoch 00000 | Time(s) nan | Loss 0.6956 |ETputs(KTEPS) nan
Epoch 00001 | Time(s) nan | Loss 0.8096 |ETputs(KTEPS) nan
Epoch 00002 | Time(s) nan | Loss 0.9065 |ETputs(KTEPS) nan
Epoch 00003 | Time(s) 0.3616 | Loss 0.5571 |ETputs(KTEPS) 1424.10
Epoch 00004 | Time(s) 0.3509 | Loss 0.4992 |ETputs(KTEPS) 1467.49
Epoch 00005 | Time(s) 0.3312 | Loss 0.5304 |ETputs(KTEPS) 1554.83
Epoch 00006 | Time(s) 0.3162 | Loss 0.5058 |ETputs(KTEPS) 1628.19
Epoch 00007 | Time(s) 0.3162 | Loss 0.4986 |ETputs(KTEPS) 1628.26
Epoch 00008 | Time(s) 0.3162 | Loss 0.4969 |ETputs(KTEPS) 1628.30
Epoch 00009 | Time(s) 0.3172 | Loss 0.4953 |ETputs(KTEPS) 1623.19
Epoch 00010 | Time(s) 0.3147 | Loss 0.4960 |ETputs(KTEPS) 1636.40
Epoch 00011 | Time(s) 0.3190 | Loss 0.4975 |ETputs(KTEPS) 1613.90
Epoch 00012 | Time(s) 0.3148 | Loss 0.4963 |ETputs(KTEPS) 1635.56
Epoch 00013 | Time(s) 0

In [106]:
# try with different setup.
#args2=dummy()
args2.dropout=0.5 #dropout probability
args2.gpu=-1
args2.lr=0.1 #learning rate
args2.n_epochs=30
args2.n_hidden=32 #number of hidden gcn units
args2.n_layers=4
args2.self_loop=False #graph self-loop 
args2.weight_decay=0.0005
#args2.dataset = CernDataset(0.70, 0.25)

graphCovNN(args2)

----Data statistics------'
      #Edges 514910
      #Classes 2
      #Train samples 22591
      #Val samples 2325
      #Test samples 7358
Epoch 00000 | Time(s) nan | Loss 0.6746 |ETputs(KTEPS) nan
Epoch 00001 | Time(s) nan | Loss 1.9232 |ETputs(KTEPS) nan
Epoch 00002 | Time(s) nan | Loss 0.9294 |ETputs(KTEPS) nan
Epoch 00003 | Time(s) 0.4402 | Loss 0.5855 |ETputs(KTEPS) 1169.68
Epoch 00004 | Time(s) 0.4263 | Loss 0.5247 |ETputs(KTEPS) 1207.83
Epoch 00005 | Time(s) 0.4215 | Loss 0.5019 |ETputs(KTEPS) 1221.56
Epoch 00006 | Time(s) 0.4205 | Loss 0.4923 |ETputs(KTEPS) 1224.55
Epoch 00007 | Time(s) 0.4184 | Loss 0.4911 |ETputs(KTEPS) 1230.71
Epoch 00008 | Time(s) 0.4221 | Loss 0.4900 |ETputs(KTEPS) 1219.80
Epoch 00009 | Time(s) 0.4134 | Loss 0.4889 |ETputs(KTEPS) 1245.55
Epoch 00010 | Time(s) 0.4138 | Loss 0.4872 |ETputs(KTEPS) 1244.23
Epoch 00011 | Time(s) 0.4136 | Loss 0.4879 |ETputs(KTEPS) 1244.87
Epoch 00012 | Time(s) 0.4092 | Loss 0.4868 |ETputs(KTEPS) 1258.43
Epoch 00013 | Time(s) 0